# <center> Cours 5 : Algorithmes de recherche

On a souvent besoin de **rechercher un élément ou sa position
dans un tableau**.

La plupart des algorithmes de recherche efficaces sont basés sur
une organisation astucieuse des données.
Par exemple, pour retrouver une carte dans un jeu, il est très utile que le jeu soit trié.

## Recherche séquentielle

La méthode de recherche la plus simple est la *recherche séquentielle*
qui consiste à examiner les éléments l'un après l'autre.
- Elle s'effectue en temps linéaire en la taille du tableau dans le pire cas.
- Elle ne nécessite pas d'avoir une structure de données triée.

La fonction `recherche_seq` ci-dessous cherche de manière séquentielle si le tableau `tab` contient la valeur `element` et retourne 
- `-1` si `element` n'est pas dans le tableau `tab` 
- l'indice de la première occurence de `element` sinon. 

In [ ]:
def recherche_seq(element, tab):
    """Retourne l'indice de la première occurence de element dans tab 
       et -1 si element n'est pas dans tab."""
    i = 0
    while i < len(tab) and tab[i] != element:
        i += 1
    if i == len(tab):
        return -1  # en cas d'échec de la recherche
    else:
        return i


nombres = [15, 21, 32, 36, 42, 45, 59, 62, 84, 87, 981]
print("Le tableau d'entiers", nombres)
nb = int(input("Quel nombre cherchez-vous ? \n\nTester avec 42 et 17.\n"))
resultat = recherche_seq(nb, nombres)
if resultat == -1:
    print("le nombre", nb, "n'est pas dans le tableau")
else:
    print("le nombre", nb, "se trouve à l'indice", resultat)

Sur ce thème : **Exercices 1 et 2, TD 5.**

## Dichotomie 

Si le tableau est trié, la *recherche par dichotomie* permet de mener une recherche beaucoup plus rapide. 

### Idée générale

Soit `tab` un tableau trié dans l'ordre croissant et `i` un indice.

- Si `element < tab[i]` alors tous les éléments "à droite" de
`tab[i]` (d'indice supérieur à `i`) sont supérieurs à `element`,
et il faut continuer la recherche parmi les éléments "à gauche" de
`tab[i]` (d'indice inférieur à `i`),
- si `element > tab[i]` alors tous les éléments "à gauche" de `tab[i]` (d'indice inférieur à `i`) sont inférieurs à `element`,
et il faut continuer la recherche parmi les éléments "à droite" de
`tab[i]` (d'indice supérieur à `i`).

En utilisant cette remarque, à chaque itération, on peut diviser par 2 la taille de la partie du tableau dans laquelle chercher `element` en comparant `element` avec le contenu de la case médiane de la partie du tableau dans laquelle se fait la recherche.

In [ ]:
def recherche_dt(element, tab):
    debut = 0
    fin = len(tab) - 1
    while debut <= fin:
        milieu = (debut + fin) // 2

        if element == tab[milieu]:  # on a trouvé l'élément
            return milieu
        elif element < tab[milieu]:
            fin = milieu - 1  # on prend la moitié gauche
        else:
            debut = milieu + 1  # on prend la moitié droite
    return -1


nombres = [15, 21, 32, 36, 42, 45, 59, 62, 84, 87, 981]
# le tableau est trié donc on peut faire une dichotomie
print("Le tableau d'entiers", nombres)
nb = int(input("Quel nombre cherchez-vous ?\n\nTester avec 59, 87, 42 et 100.\n"))
resultat = recherche_dt(nb, nombres)
if resultat == -1:
    print("le nombre", nb, "n'est pas dans le tableau")
else:
    print("le nombre", nb, "se trouve à l'indice", resultat)

Sur ce thème : **Exercice 3, TD 5.**

### Complexité asymptotique

La complexité asymptotique de la recherche par dichotomie dépend du nombre d'itérations de la boucle.

**Proposition :** La recherche par dichotomie se déroulant en au plus $k$ itérations permet de chercher une valeur dans un tableau trié de taille au plus $2^k-1$. 

**Preuve** par récurrence.
- Vrai pour $k = 1$ car en 1 test, on ne peut chercher que dans un tableau de taille 1,
- Si c'est vrai pour $k$, alors en $k+1$ itérations, on peut faire la recherche sur un tableau pour lequel la recherche par dichotomie sur les parties gauche et droite du tableau se fait en au plus $k$ itérations.
    
    ![alt](img/dicho.svg)

    Les parties gauche et droite ont une taille d'au plus $2^k - 1$ (par hypothèse de récurrence) donc la taille totale du tableau est au plus : $(2^k-1) + 1 + (2^k-1) = 2^{k+1}-1$.


Dans le pire cas, pour un tableau de taille $n$, le nombre d'itérations nécessaires est donc égal au plus petit entier $k$ tel que $n \leq 2^k-1$, soit $k = \lceil log_2(n+1) \rceil = \lfloor \log_2(n) \rfloor + 1$. 

Comme le nombre d'opérations élémentaires par itération est borné par une constante, la complexité asymptotique de la recherche par dichotomie dans un tableau de taille $n$ est $\mathcal{O}(\log_2(n))$, soit une **complexité asymptotique logarithmique**.

**Remarque :** La recherche dichotomique est plus rapide que la recherche séquentielle en général (complexité en moyenne et dans le pire des cas) mais dans certains cas, la recherche séquentielle peut être plus rapide : si les tableaux sont de petite taille, ou si la valeur à chercher se trouve en début de tableau (exemple rechercher 15 dans le tableau `nombres`).

## Courbes de complexité

In [ ]:
from matplotlib.pyplot import plot, figure, legend, xlabel, ylabel
from time import time
from random import randint

In [ ]:
def cree_tab(n):
    """Retourne le tableau [0, 1, ..., n-1]"""
    t = []
    i = 0
    while i < n:
        t.append(i)
        i += 1
    return t


def mesure_temps_recherche(n, nb_mesures):
    """Retourne le temps moyen (sur nb_mesures) d'une recherche d'un élément 
    dans un tableau trié. Le tableau retourné contient le temps pour une 
    recherche séquentielle (indice 0) puis dichotomique (indice1)."""
    tps = [0., 0.]
    tableau = cree_tab(n)
    i = 0
    while i < nb_mesures:
        element = n

        # Recherche séquentielle
        tic = time()
        res = recherche_seq(element, tableau)
        tac = time()
        tps[0] += tac - tic

        # Recherche dichotomique
        tic = time()
        res = recherche_dt(element, tableau)
        tac=time()
        tps[1] += tac - tic

        i += 1

    tps[0] = round(1000 * tps[0] / nb_mesures, 6)
    tps[1] = round(1000 * tps[1] / nb_mesures, 6)
    return tps


# Mesures des temps :
# - temps_seq[i] contient le temps moyen de recherche séquentielle pour un tableau de taille tailles[i]
# - temps_dt[i] contient le temps moyen de recherche dichotomique pour un tableau de taille tailles[i]
nb_mesures = 20
tailles = [100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000]
temps_seq = []
temps_dt = []
i = 0
while i < len(tailles):
    tps = mesure_temps_recherche(tailles[i], nb_mesures)
    temps_seq.append(tps[0])
    temps_dt.append(tps[1])
    i += 1

In [ ]:
print("Temps moyen de recherche :")
i = 0
while i < len(tailles):
    print("Pour un tableau de taille ", tailles[i], ":")
    print("\t", temps_seq[i], "ms pour la recherche séquentielle")
    print("\t", temps_dt[i], "ms pour la recherche dichotomique")
    i += 1

In [ ]:
%matplotlib inline

plot(tailles, temps_seq, label="séquentiel")
plot(tailles, temps_dt, label="dichotomie")
xlabel("taille")
ylabel("temps (ms)")
legend()

Sur ce thème : **Exercice 4, TD 5.**